In [ ]:
# ------------------------------------------------------------
# 2  BL Solver with max excess return as fallback
# ------------------------------------------------------------
from pypfopt import EfficientFrontier
from pypfopt.exceptions import OptimizationError

def ewm_covariance(returns, halflife=126, min_periods=60):
    ewm_cov = returns.ewm(halflife=halflife,
                          adjust=False,
                          min_periods=min_periods).cov()
    if returns.empty: return pd.DataFrame()
    return ewm_cov.loc[returns.index[-1]]

def detect_state_shifts(views, factors):
    # 1 col per factor with the model‑state
    state_df = pd.concat({f: views[f]["state"] for f in factors}, axis=1)
    # True when *any* factor changes state vs. the day before
    return state_df.ne(state_df.shift()).any(axis=1)

# ---------- helper: dict -> (P,Q) for relative views ----------
def make_relative_views(view_dict, assets, benchmark="Market"):
    """
    Convert {'fac': active_ret, ...} into (P,Q) so that
    E[fac] - E[benchmark] = active_ret  for every factor.
    """
    if benchmark not in assets:
        raise ValueError(f"Benchmark {benchmark!r} not in trade universe")

    n = len(assets)
    k = len(view_dict)
    P = np.zeros((k, n))
    Q = np.zeros(k)

    for i, (fac, v) in enumerate(view_dict.items()):
        if fac not in assets:
            continue                      # ignore missing factors
        P[i, assets.index(fac)]       = 1
        P[i, assets.index(benchmark)] = -1
        Q[i] = v                       # the expected active return
    return P, Q


def bl_max_sharpe_te(cov_hist, pi, views, tau, delta,
                     w_bmk, te_target, bounds, rf,
                     fallback_strategy,
                     use_bl_cov=False):

    """
    Returns target weights *over trade_assets* (no cash leg).
    Falls back to the CVXPY excess-return solver if max_sharpe errors.
    """
    # 1) Black-Litterman → implied mu & Sigma
    P, Q = make_relative_views(views, list(w_bmk.index), benchmark="Market")
    bl = BlackLittermanModel(cov_hist, pi=pi, tau=tau,
                             delta=delta, P=P, Q=Q)
    Sigma = bl.bl_cov() if use_bl_cov else cov_hist
    mu    = bl.bl_returns()
    n     = len(mu)
    bmk_w = w_bmk.values
    Σ     = Sigma.values

    # 2) set up EF + TE + bounds
    ef = EfficientFrontier(mu, Sigma)
    ef.add_constraint(lambda w: cp.quad_form(w - bmk_w, Σ) <= te_target**2)
    for i, (lo, hi) in enumerate(bounds):
        ef.add_constraint(lambda w, i=i, lo=lo: w[i] >= lo)
        ef.add_constraint(lambda w, i=i, hi=hi: w[i] <= hi)

    # 3) try max_sharpe, else fallback to CVXPY
    try:
        raw_w = ef.max_sharpe(risk_free_rate=rf)
    except (OptimizationError, ValueError):
        w_var = cp.Variable(n)
        w_act = w_var - bmk_w
        prob = cp.Problem(
            cp.Maximize((mu.values - rf) @ w_var),
            [
                cp.sum(w_var) == 1,
                w_var >= np.array([lo for lo, hi in bounds]),
                w_var <= np.array([hi for lo, hi in bounds]),
                cp.quad_form(w_act, Σ) <= te_target**2
            ]
        )
        prob.solve(solver="SCS")
        raw_w = dict(zip(w_bmk.index, w_var.value))

    return pd.Series(raw_w, index=w_bmk.index)


def run_bl_with_drift(views, returns_df, full_df,
                      shift_series=None,
                      tau=0.05, delta=2.5,
                      te_target=0.05,
                      trade_market=True,
                      use_bl_cov=False,
                      allow_market_short=False,
                      allow_factor_short=False,
                      use_bl_prior=False,
                      fallback_strategy="HOLD_RFR",
                      tcost=0.0007,
                      initial_capital=1_000_000):
    """
    Returns:
      w_view   -> DataFrame of *target* weights from the optimizer
      w_actual -> DataFrame of *actual* daily weights, drifting unless rebalance
      rets     -> Series of daily portfolio returns (in decimal, e.g. 0.01 = 1%)
    """

    assets  = returns_df.columns.tolist()
    factors = list(views.keys())

    # ——————————————
    #  Per‐factor regime‐shift flags
    state_df = pd.concat({f: views[f]["state"] for f in factors}, axis=1)
    per_factor_shifts = (
        state_df.ne(state_df.shift())            # True where each factor flips
        .reindex(returns_df.index, fill_value=False)
    )
    per_factor_shifts.iloc[0, :] = True         # force update on day 1

    # Initialize “last seen” active_ret for each factor
    prev_active_ret = {
        fac: views[fac].loc[returns_df.index[0], "active_ret"]
        for fac in factors
    }
    # ——————————————


    # Decide which assets can be traded
    if trade_market:
        trade_assets = [a for a in assets if a != "rf"]
    else:
        trade_assets = [a for a in assets if a not in {"rf", "Market"}]

    cash_asset = "rf"

    # Bounds
    bounds = []
    for a in trade_assets:
        if a == "Market":
            bounds.append((-.3, .3) if allow_market_short else (0, 0.3))
        else:
            bounds.append((-.3, .3) if allow_factor_short else (0, 0.3))

    # Prepare DataFrames
    w_view   = pd.DataFrame(0.0, index=returns_df.index, columns=trade_assets + [cash_asset], dtype=float)
    w_actual = pd.DataFrame(0.0, index=returns_df.index, columns=trade_assets + [cash_asset], dtype=float)

    # Track positions in “dollar” terms
    positions = pd.Series(0.0, index=trade_assets + [cash_asset])
    capital   = initial_capital

    # Series of daily returns, e.g. +0.005 = +0.5%
    rets = pd.Series(0.0, index=returns_df.index)

    for i, t in enumerate(returns_df.index):
        old_capital = capital  # Store capital before today’s returns

        # 1) Realize today's P&L from yesterday’s positions
        if i > 0:
            day_ret = returns_df.loc[t, trade_assets + [cash_asset]]
            daily_pnl = (positions[trade_assets] * day_ret[trade_assets]).sum() \
                        + positions[cash_asset] * day_ret[cash_asset]

            capital += daily_pnl
            # convert P&L → daily return = P&L / yesterday’s capital
            rets.loc[t] = daily_pnl / old_capital

        # 2) Decide if we rebalance today
        do_rebalance = False
        if i == 0:
            do_rebalance = True
        elif shift_series is not None and shift_series.loc[t]:
            do_rebalance = True

        # 3) If rebalancing, solve for new “view” weights
        if do_rebalance:
            hist = full_df[trade_assets].loc[:t].iloc[:-1]
            cov  = ewm_covariance(hist) * 252

            if cov.empty or cov.isna().any().any():
                w_view.loc[t, trade_assets] = 0.0
                w_view.loc[t, cash_asset]   = 1.0  # 100% cash
            else:
                # Build a quick "light" BlackLittermanModel
                if use_bl_prior:
                    market_caps  = {etf: 1.0 for etf in trade_assets}
                    prior_for_bl = market_implied_prior_returns(market_caps, delta, cov)
                else:
                    prior_for_bl = "equal"

                q = {}
                for fac in factors:
                    if per_factor_shifts.loc[t, fac]:
                        q_val = views[fac].loc[t, "active_ret"]
                    else:
                        q_val = prev_active_ret[fac]
                    q[fac] = q_val
                    prev_active_ret[fac] = q_val
                rf_annual = returns_df.loc[t, cash_asset] * 252

                # Build a quick "light" BlackLittermanModel
                q_abs = {fac: views[fac].loc[t, "ann_abs_ret"] for fac in factors}

                bl0 = BlackLittermanModel(
                    cov,
                    pi=prior_for_bl,
                    tau=tau,
                    delta=delta,
                    absolute_views=q_abs
)

                # 2) Check fallback if all implied returns ≤ rf
                if (
                    fallback_strategy != "NO_FALLBACK"
                    and (bl0.bl_returns() <= rf_annual).all()
                ):
                    # “Risk-off” scenario
                    w_view.loc[t, trade_assets] = 0.0  # all 0
                    if fallback_strategy == "HOLD_RFR":
                        w_view.loc[t, cash_asset] = 1.0  # 100% in cash
                    elif fallback_strategy == "SHORT_MARKET" and "Market" in trade_assets:
                        w_view.loc[t, "Market"]   = -1.0
                        w_view.loc[t, cash_asset] = 1.0
                    # skip the main BL max-sharpe
                else:
                    # normal BL
                    w_opt = bl_max_sharpe_te(
                        cov,
                        pi=prior_for_bl,
                        views=q,
                        tau=tau,
                        delta=delta,
                        w_bmk=pd.Series(1 / len(trade_assets), index=trade_assets),
                        te_target=te_target,
                        bounds=bounds,
                        rf=rf_annual,
                        use_bl_cov=use_bl_cov,
                        fallback_strategy=fallback_strategy
                    )
                    w_view.loc[t, trade_assets] = w_opt
                    w_view.loc[t, cash_asset]   = 1 - w_opt.sum()


            # 4) Convert new “view weights” → actual $ positions
            new_positions = capital * w_view.loc[t]

            # transaction cost
            if tcost > 0 and i > 0:
                asset_trades = (new_positions[trade_assets] - positions[trade_assets]).abs().sum()
                cost        = asset_trades * tcost
                capital    -= cost
                rets.loc[t] = (capital - old_capital) / old_capital

            positions = new_positions

        else:
            # Not rebalancing → copy forward yesterday’s “view” weights
            if i > 0:
                w_view.loc[t] = w_view.iloc[i - 1]

        # 5) Actual weights
        if capital > 0:
            w_actual.loc[t] = positions / capital
        else:
            w_actual.loc[t] = 0.0

    return w_view, w_actual, rets


In [ ]:
# ------------------------------------------------------------
# 2  Max excess return with TE constraint - falls at the max sharpe portfolio 
# ------------------------------------------------------------

def ewm_covariance(returns, halflife=126, min_periods=60):
    ewm_cov = returns.ewm(halflife=halflife,
                          adjust=False,
                          min_periods=min_periods).cov()
    if returns.empty: return pd.DataFrame()
    return ewm_cov.loc[returns.index[-1]]

def detect_state_shifts(views, factors):
    # 1 col per factor with the model‑state
    state_df = pd.concat({f: views[f]["state"] for f in factors}, axis=1)
    # True when *any* factor changes state vs. the day before
    return state_df.ne(state_df.shift()).any(axis=1)

# ---------- helper: dict -> (P,Q) for relative views ----------
def make_relative_views(view_dict, assets, benchmark="Market"):
    """
    Convert {'fac': active_ret, ...} into (P,Q) so that
    E[fac] - E[benchmark] = active_ret  for every factor.
    """
    if benchmark not in assets:
        raise ValueError(f"Benchmark {benchmark!r} not in trade universe")

    n = len(assets)
    k = len(view_dict)
    P = np.zeros((k, n))
    Q = np.zeros(k)

    for i, (fac, v) in enumerate(view_dict.items()):
        if fac not in assets:
            continue                      # ignore missing factors
        P[i, assets.index(fac)]       = 1
        P[i, assets.index(benchmark)] = -1
        Q[i] = v                       # the expected active return
    return P, Q


def bl_max_sharpe_te(cov_hist, pi, views, tau, delta,
                     w_bmk, te_target, bounds, rf,
                     use_bl_cov=False):

    P, Q = make_relative_views(views, list(w_bmk.index), benchmark="Market")
    bl   = BlackLittermanModel(cov_hist, pi=pi, tau=tau,
                           delta=delta, P=P, Q=Q)

    # pick the risk matrix
    Sigma = bl.bl_cov().values if use_bl_cov else cov_hist.values
    mu    = bl.bl_returns().values
    n     = len(mu)

    w = cp.Variable(n)
    w_act = w - w_bmk.values

    prob = cp.Problem(
        cp.Maximize((mu - rf) @ w),
        [
            cp.sum(w) == 1,
            w >= np.array([lo for lo, hi in bounds]),
            w <= np.array([hi for lo, hi in bounds]),
            cp.quad_form(w_act, Sigma) <= te_target**2
        ]
    )
    prob.solve(solver="SCS")
    return pd.Series(w.value, index=w_bmk.index)


def run_bl_with_drift(views, returns_df, full_df,
                      shift_series=None,
                      tau=0.05, delta=2.5,
                      te_target=0.05,
                      trade_market=True,
                      use_bl_cov=False,
                      allow_market_short=False,
                      allow_factor_short=False,
                      use_bl_prior=False,
                      fallback_strategy="HOLD_RFR",
                      tcost=0.0007,
                      initial_capital=1_000_000):
    """
    Returns:
      w_view   -> DataFrame of *target* weights from the optimizer
      w_actual -> DataFrame of *actual* daily weights, drifting unless rebalance
      rets     -> Series of daily portfolio returns (in decimal, e.g. 0.01 = 1%)
    """

    assets  = returns_df.columns.tolist()
    factors = list(views.keys())

    # ——————————————
    #  Per‐factor regime‐shift flags
    state_df = pd.concat({f: views[f]["state"] for f in factors}, axis=1)
    per_factor_shifts = (
        state_df.ne(state_df.shift())            # True where each factor flips
        .reindex(returns_df.index, fill_value=False)
    )
    per_factor_shifts.iloc[0, :] = True         # force update on day 1

    # Initialize “last seen” active_ret for each factor
    prev_active_ret = {
        fac: views[fac].loc[returns_df.index[0], "active_ret"]
        for fac in factors
    }
    # ——————————————


    # Decide which assets can be traded
    if trade_market:
        trade_assets = [a for a in assets if a != "rf"]
    else:
        trade_assets = [a for a in assets if a not in {"rf", "Market"}]

    cash_asset = "rf"

    # Bounds
    bounds = []
    for a in trade_assets:
        if a == "Market":
            bounds.append((-.3, .3) if allow_market_short else (0, 0.3))
        else:
            bounds.append((-.3, .3) if allow_factor_short else (0, 0.3))

    # Prepare DataFrames
    w_view   = pd.DataFrame(0.0, index=returns_df.index, columns=trade_assets + [cash_asset], dtype=float)
    w_actual = pd.DataFrame(0.0, index=returns_df.index, columns=trade_assets + [cash_asset], dtype=float)

    # Track positions in “dollar” terms
    positions = pd.Series(0.0, index=trade_assets + [cash_asset])
    capital   = initial_capital

    # Series of daily returns, e.g. +0.005 = +0.5%
    rets = pd.Series(0.0, index=returns_df.index)

    for i, t in enumerate(returns_df.index):
        old_capital = capital  # Store capital before today’s returns

        # 1) Realize today's P&L from yesterday’s positions
        if i > 0:
            day_ret = returns_df.loc[t, trade_assets + [cash_asset]]
            daily_pnl = (positions[trade_assets] * day_ret[trade_assets]).sum() \
                        + positions[cash_asset] * day_ret[cash_asset]

            capital += daily_pnl
            # convert P&L → daily return = P&L / yesterday’s capital
            rets.loc[t] = daily_pnl / old_capital

        # 2) Decide if we rebalance today
        do_rebalance = False
        if i == 0:
            do_rebalance = True
        elif shift_series is not None and shift_series.loc[t]:
            do_rebalance = True

        # 3) If rebalancing, solve for new “view” weights
        if do_rebalance:
            hist = full_df[trade_assets].loc[:t].iloc[:-1]
            cov  = ewm_covariance(hist) * 252

            if cov.empty or cov.isna().any().any():
                w_view.loc[t, trade_assets] = 0.0
                w_view.loc[t, cash_asset]   = 1.0  # 100% cash
            else:
                # Build a quick "light" BlackLittermanModel
                if use_bl_prior:
                    market_caps  = {etf: 1.0 for etf in trade_assets}
                    prior_for_bl = market_implied_prior_returns(market_caps, delta, cov)
                else:
                    prior_for_bl = "equal"

                q = {}
                for fac in factors:
                    if per_factor_shifts.loc[t, fac]:
                        q_val = views[fac].loc[t, "active_ret"]
                    else:
                        q_val = prev_active_ret[fac]
                    q[fac] = q_val
                    prev_active_ret[fac] = q_val
                rf_annual = returns_df.loc[t, cash_asset] * 252

                # Build a quick "light" BlackLittermanModel
                q_abs = {fac: views[fac].loc[t, "ann_abs_ret"] for fac in factors}

                bl0 = BlackLittermanModel(
                    cov,
                    pi=prior_for_bl,
                    tau=tau,
                    delta=delta,
                    absolute_views=q_abs
)

                # 2) Check fallback if all implied returns ≤ rf
                if (
                    fallback_strategy != "NO_FALLBACK"
                    and (bl0.bl_returns() <= rf_annual).all()
                ):
                    # “Risk-off” scenario
                    w_view.loc[t, trade_assets] = 0.0  # all 0
                    if fallback_strategy == "HOLD_RFR":
                        w_view.loc[t, cash_asset] = 1.0  # 100% in cash
                    elif fallback_strategy == "SHORT_MARKET" and "Market" in trade_assets:
                        w_view.loc[t, "Market"]   = -1.0
                        w_view.loc[t, cash_asset] = 1.0
                    # skip the main BL max-sharpe
                else:
                    # normal BL
                    w_opt = bl_max_sharpe_te(
                        cov,
                        pi=prior_for_bl,
                        views=q,
                        tau=tau,
                        delta=delta,
                        w_bmk=pd.Series(1 / len(trade_assets), index=trade_assets),
                        te_target=te_target,
                        bounds=bounds,
                        rf=rf_annual,
                        use_bl_cov=use_bl_cov
                    )
                    w_view.loc[t, trade_assets] = w_opt
                    w_view.loc[t, cash_asset]   = 1 - w_opt.sum()


            # 4) Convert new “view weights” → actual $ positions
            new_positions = capital * w_view.loc[t]

            # transaction cost
            if tcost > 0 and i > 0:
                asset_trades = (new_positions[trade_assets] - positions[trade_assets]).abs().sum()
                cost        = asset_trades * tcost
                capital    -= cost
                rets.loc[t] = (capital - old_capital) / old_capital

            positions = new_positions

        else:
            # Not rebalancing → copy forward yesterday’s “view” weights
            if i > 0:
                w_view.loc[t] = w_view.iloc[i - 1]

        # 5) Actual weights
        if capital > 0:
            w_actual.loc[t] = positions / capital
        else:
            w_actual.loc[t] = 0.0

    return w_view, w_actual, rets
